# Cyclopeptide Sequencing Algorithm

- Aassume that in the mass table the key of the value 113, 128, always are L, Q respectively

### Add the Spectrum Map

In [7]:
mass_table = {'G': 57, 'A': 71, 'S': 87, 'P': 97, 'V': 99, 'T': 101,
              'C': 103, 'I': 113, 'L': 113, 'N': 114, 'D': 115,
              'K': 128, 'Q': 128, 'E': 129, 'M': 131, 'H': 137,
              'F': 147, 'R': 156, 'Y': 163, 'W': 186}

# Invert the mass table to map mass to amino acid symbol
mmap = {value: key for key, value in mass_table.items()}

### Initialize the List of Amino Acids

In [8]:
def initial_list(spect):
  """
  This function takes a list of spectrum values (masses) and returns a list of
  corresponding amino acid symbols from the mass_table, considering a mass threshold.

  Args:
      spect: A list of integers representing spectrum values (masses).

  Returns:
      A list containing amino acid symbol for elements
      in the spectrum that have a mass less than or equal to 186.
  """
  matching_data = [ mmap[val] for val in spect if val <= 186]
  return matching_data


### Generate k-mers

In [9]:
def generate_n_mers(letters):
    """
    This function create a list of the next k-mer level based on the given list

    Input: list of k-mer and the initial list which will be used in extended the given list

    Output: A list of the next k-mer.
    """
    n = 2
    n_minus_1_mers = letters

    def generate():
        nonlocal n, n_minus_1_mers
        if n == 1:
            return n_minus_1_mers
        else:
            n_mers = []
            for mer in n_minus_1_mers:
                for letter in letters:
                    if letters.count(letter) > mer.count(letter):
                          n_mers.append(mer + letter)

            n_minus_1_mers = n_mers
            return list(set(n_mers))

    while True:
        yield generate()
        n += 1


### Generate List of Subpeptides & Calculate Mass of Each Subpeptide

In [ ]:
def GenerateSubpeptides(peptide) :
    """
    This function generate subpeptides of the given peptide.

    Input: peptide that will be splitted into different subpeptides.

    Output: A list of subpeptides.
    """
    subpeptides = []
    subpp = ''
    for i in range(0,len(peptide),1):   
        mer_size = i+1                      
        for j in range(0,len(peptide),1): 
            subpp = peptide[j:j + mer_size] 
            if (len(subpp) < mer_size):  
                continue
            subpeptides.append(subpp)         
        if mer_size == len(peptide):      
            break
        subpp=''
    return subpeptides

def CalculatePeptide (peptide): 
    """
    Calculate the total mass of every subpeptide within a given peptide sequence.

    Input: peptide (str), The peptide sequence for which the total mass of subpeptides is calculated.

    Output:  The total mass of all subpeptides in the peptide sequence.
    """   
    Total=0
    for i in peptide:
        Total += mass_table[i]
    return Total

### Find the Linear Spectrum

In [77]:
def Linear_Spectrum(peptide) :
    """
    Calculate the linear spectrum of a peptide sequence.

    Input: peptide (str), The peptide sequence for which the linear spectrum is calculated.

    Output: A list of integers representing the linear spectrum of the peptide.
    """
    SpectrumList = []  
    SubpeptidesList=[]

    SubpeptidesList = GenerateSubpeptides(peptide)
    for subpp in SubpeptidesList:
        SpectrumList.append(CalculatePeptide(subpp))
    SpectrumList.sort()
    
    return (SpectrumList)

### Check If The Mer is Consistent

In [78]:
def is_consistent(peptide, spectrum):    """
Description:This function Determines whether a subpeptide is consistent with a given spectrum. A subpeptide isconsidered consistent if its linear spectrum, derived from the masses of its amino acids, is entirelycontained within the spectrum.Parameters:   - subpeptide (str): The subpeptide sequence to be checked for consistency.  - spectrum (list): The theoretical spectrum against which the consistency of the subpeptide is  evaluated.Returns:      bool: True if the subpeptide is consistent with the spectrum, False otherwise.    """
 peptide_spectrum = Linear_Spectrum(peptide)for mass in peptide_spectrum:   if mass not in spectrum:      return Falsereturn True

### Loop That Combine All the Prevoius Functions (Cyclopeptide_Sequencing)

In [79]:
def Cyclopeptide_Sequencing(t_spect):
    """
    This function performs Cyclopeptide Sequencing to identify cyclic peptides that align with the provided spectrum.

    Input: spectrum (list), The theoretical spectrum against which the peptides are evaluated.

    Output: A list containing cyclic peptide sequences that are consistent with the provided spectrum.
    """
    spect_filtered = [val for val in t_spect if val > 0] 
    initial = initial_list(spect_filtered)

    flag = True
    temp = generate_n_mers(initial)

    while flag:
    
        final_list=[]
        tmp=next(temp)
        
        for peptide in tmp:
                con_flag = is_consistent(peptide,t_spect)
                if(con_flag == True):
                    final_list.append(peptide)
                elif (con_flag ==False):
                    tmp.remove(peptide)
        
        if len(final_list) == 0:
                flag = False
        else:
                final =final_list 
    
    return final

In [80]:
 therorical_spectrum = [int(x) for x in input(Enter The Therorical Spectrum: ).split()]seq = Cyclopeptide_Sequencing(therorical_spectrum)print (The Seq. are: , seq)#Test Case:#0 97 97 99 101 103 196 198 198 200 202 295 297 299 299 301 394 396 398 400 400 497

The Seq. are:  ['CPTPV', 'PTVPC', 'PTPCV', 'CPTVP', 'TPVCP', 'TVPCP', 'VPTPC', 'PVTPC', 'PCPVT', 'CVPTP', 'VCPTP', 'PTPVC']
